# INSTRUCTIONS ON RUNNING COLAB ASSIGNMENTS

1. Run every cell. (You can skip the text cells, but might as well run them anyways.) _Especially_ the `import` cells -- which determine the python packages and libraries that your colab has access to. To run a cell and move to the next one, press `shift + enter`.
2. BE CAREFUL RUNNING CELLS OUT OF ORDER -- if you have modified a variable, you will be using the latest version of that variable, regardless of the order of the cells in the notebook!
3. If you've been editing for a while and are getting a weird error, you might try restarting the notebook and running all the cells (in order). Click on `Runtime -> Restart session and run all.`

# Cart Pole Simulation

We have implemented a cart-pole simulation using forward-euler integration to simulate the dynamics of the cart-pole using the equations of motion derived in the lecture.

### Your Task:

1. Without any control input, the pendulum will fall to the swing-down position. It is your task to tune the appropriate PID gains: proportional, integral, and derivative.
2. (Later) Tune the cost matrix of an LQR controller to see how your PID controller compares to an _optimal controller_.

In [ ]:
# --- NECESSARY IMPORTS ---
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
# --- ANIMATION IMPORTS ---
import os
!pip install pygame > /dev/null 2>&1 # supress output
!pip3 install numpngw > /dev/null 2>&1 # supress output
!pip3 install IPython > /dev/null 2>&1 # supress output
import pygame
import numpngw
from IPython.display import Image, display

We define a PIDController Class. This contains an **update** function which computes the force from the PID gains you provide.

We also define a CartPoleSystem, which steps through the equations of motion -- applying the physical forces, as well as our control force.

In [ ]:
# --- CLASS DEFINITIONS ---
# PID controller
class PIDController:
  def __init__(self, kp, ki, kd, dt):
    self.kp = kp
    self.ki = ki
    self.kd = kd
    self.dt = dt
    self.integral = 0

  def update(self, setpoint, state):
    # Note that the sign of this error is a little unintuitive
    error_vec = setpoint - state
    error = error_vec[2] # qB error
    derivative = error_vec[3] # dqB error
    self.integral += error * self.dt
    # Note influence of ki term depends a lot on timestep size
    output = self.kp * error + self.ki * self.integral + self.kd * derivative
    return output #TODO: check sign

# Cart pole system
class CartPoleSystem:
  def __init__(self, L, mA, mB, bt, br, g, IBxx, IByy, IBzz, controller, F_max = 10):

    # Dynamics Constants
    self.L = L
    self.mA = mA
    self.mB = mB
    self.bt = bt
    self.br = br
    self.g = g
    self.IBxx = IBxx
    self.IByy = IByy
    self.IBzz = IBzz
    # Force saturation
    self.F_max = F_max
    # Tracking other parameters
    self.F_history = []

    self.controller = controller

  def cart_pole_dynamics(self, t, y):
    x, dx, qB, dqB = y
    state = np.array(y)
    setpoint = np.zeros_like(state)
    # Define control force
    F = self.controller.update(setpoint, state)
    # Runtime assurance
    F = np.clip(F, -self.F_max, self.F_max)
    self.F_history.append((t,F))
    # Translational equation of motion
    ddx = -0.5*(2*self.L*self.mB*np.cos(qB)*(g*self.L*self.mB*np.sin(qB)-2*self.br*dqB)+(4*self.IByy+self.mB*self.L**2)
          *(2*self.bt*dx-2*F-self.L*self.mB*np.sin(qB)*dqB**2))/((self.mA+self.mB)*(4*self.IByy+self.mB*self.L**2)
          -self.L**2*self.mB**2*np.cos(qB)**2)
    # Rotational equation of motion
    ddqB = (2*(self.mA+self.mB)*(self.g*self.L*self.mB*np.sin(qB)-2*self.br*dqB)+self.L*self.mB*np.cos(qB)
          *(2*self.bt*dx-2*F-self.L*self.mB*np.sin(qB)*dqB**2))/((self.mA+self.mB)*(4*self.IByy+self.mB*self.L**2)
          -self.L**2*self.mB**2*np.cos(qB)**2)
    # Return derivatives
    return [dx, ddx, dqB, ddqB]

In [ ]:
#@title Initialize the Simulation
# Dynamics constants
L = 0.60 # m
mA = 0.261 # kg
mB = 0.351 # kg
bt = 0.2 # N*s/m
br = 0.0015 # N*m*s/rad
g = 9.81 # m/s^2
IBxx = 1/12*mB*L**2 # kg*m^2
IByy = IBxx # kg*m^2
IBzz = 0 # kg*m^2

# Simulation Constants
t_end = 4
dt = 0.005
n_steps = int(t_end/dt) + 1
t_eval = np.linspace(0, t_end, n_steps)

In [ ]:
#@title Plotting and Animation Code Helper Functions (RUN ME)

# --- PLOTTING ---
def plot_cart_pole(PLOT_LQR = False):
    """Helper function for cart-pole plotting with options for LQR."""
    # --- PLOTTING ---
    vars = ['x','v','theta','omega']
    units = ['m','m/s','rad','rad/s']

    # Plot state variables
    for i in range(sol_pid.y.shape[0]):
      plt.figure()
      if PLOT_LQR:
          try:
              plt.plot(sol_lqr.t, sol_lqr.y[i, :], label='LQR')
          except NameError:
              print("`sol_lqr` is not defined, so it cannot be plotted. Have you run the LQR code blocks?")
              PLOT_LQR = False
      plt.plot(sol_pid.t, sol_pid.y[i,:], label='PID')
      plt.legend()

      plt.title(f'Cart Pole: {vars[i]}')
      plt.xlabel(f't (seconds)')
      plt.ylabel(f'{vars[i]} ({units[i]})')

    # Plot control force
    plt.figure()
    ctrl_t, ctrl_F = zip(*cart_pole_pid.F_history)
    ctrl_F_at_eval = interp1d(ctrl_t, ctrl_F, kind='nearest', fill_value="extrapolate")(sol_pid.t).squeeze()
    plt.plot(sol_pid.t, ctrl_F_at_eval, label='PID')
    if PLOT_LQR:
      ctrl_t_lqr, ctrl_F_lqr = zip(*cart_pole_lqr.F_history)
      ctrl_F_lqr_at_eval = interp1d(ctrl_t_lqr, ctrl_F_lqr, kind='nearest', fill_value="extrapolate")(sol_lqr.t).squeeze()
      plt.plot(sol_lqr.t, ctrl_F_lqr_at_eval, label='LQR')
      plt.legend()
    plt.title('Cart Pole: Control Force')
    plt.xlabel(f't (seconds)')
    plt.ylabel(f'F (N)')

# --- ANIMATION ---
# Define animation timestep
animation_dt = 0.075

# Delete old file
animation_path = 'cart_pole.gif'
if os.path.exists(animation_path):
  os.remove(animation_path)

# Screen settings
scale = 200
screen_width = 2 # m
screen_height = 1.5 # m
screen_width_px = int(screen_width*scale)
screen_height_px = int(screen_height*scale)

# Colors
gray = (125, 125, 125)
black = (0, 0, 0)
white = (255, 255, 255)
red = (255, 0, 0)

# Cart settings
cart_color = (125, 125, 125)
cart_width = 0.2 # m
cart_height = 0.15 # m

# Rail settings
rail_width = 0.02 # m
rail_length = 1.8 # m

# Pole settings
pole_length = L # m
pole_width_px = 3

# Screen origin
origin_x = screen_width/2
origin_y = screen_height/2

# Scaling
cart_width_px = int(cart_width*scale)
cart_height_px = int(cart_height*scale)
rail_width_px = int(rail_width*scale)
rail_length_px = int(rail_length*scale)
pole_length_px = int(pole_length*scale)
origin_x_px = int(origin_x*scale)
origin_y_px = int(origin_y*scale)

# Function to convert Pygame surface to an image (numpy array)
def pygame_to_img(surface):
    return pygame.surfarray.array3d(surface).transpose([1, 0, 2])

# Function to draw rectangle at its center
def draw_center_rect(screen, x_c, y_c, width, height, color):
  x = x_c - width/2
  y = y_c - height/2
  pygame.draw.rect(screen, color, (x, y, width, height))

def animate_system(soln, n_steps, dt, animation_dt):
  # Initialize Pygame
  pygame.init()

  # Calculate amount of frames in animation
  n_frames = int(dt/animation_dt*n_steps)

  # List to store numpy array images
  np_frames = []
  for frame_number in range(n_frames):
    # Calculate time index
    time_idx = int(animation_dt/dt*frame_number)

    # Get cart position
    x_cart = soln.y[0,time_idx]
    y_cart = origin_y

    # Calculate pole endpoint
    x_pole = x_cart + pole_length*np.sin(soln.y[2,time_idx])
    y_pole = y_cart + pole_length*np.cos(soln.y[2,time_idx])

    # Scale coordinates
    x_cart_px = int(x_cart*scale)
    y_cart_px = int(y_cart*scale)
    x_pole_px = int(x_pole*scale)
    y_pole_px = int(y_pole*scale)

    # Fill screen with white background
    screen.fill(white)

    # Draw rail
    draw_center_rect(screen, origin_x_px, screen_height_px - origin_y_px, rail_length_px, rail_width_px, black)

    # Draw cart
    draw_center_rect(screen, origin_x_px - x_cart_px, screen_height_px - origin_y_px, cart_width_px, cart_height_px, gray)

    # Draw pole
    pygame.draw.line(screen, red, (origin_x_px - x_cart_px, screen_height_px - y_cart_px), (origin_x_px - x_pole_px, screen_height_px - y_pole_px), pole_width_px)

    # Save frame
    np_frames.append(pygame_to_img(screen))

  # Quit Pygame only after all frames are processed
  pygame.quit()
  return np_frames


## Your Turn - Tune the PID gains!
Next, we initialize our system with the correct values and initial conditions. This is where you come in! Select the PID gains that stabilize the system.

Some helpful tips for tuning:
- Start with P-gain (keeping the others zero). Increase the P-gain until you see the system respond with enough force to counteract gravity and bring the pendulum back to the top. With only proportional gain, the system will overshoot wildly, not settling. A physical analogy for P-gain is a "spring" (remember Hooke's' law: $F = -k x$? That's like P-gain!) The more P, the bouncier the system will behave.
- Next, increase D-gain. An analogy for D-gain is "damping". (Friction is often modeled as $F = -k v$.) D-gain helps the system slow down as the pendulum approaches the setpoint to avoid overshooting. Increase D-gain until you see this effect.
- Finally, increase I-gain. I-gain helps eliminate residual error by nudging the system to the correct position even if the error is not large enough for P-gain to take sufficient action. Since I-gain requires integration over time, it will not act as quickly as P-gain. Strike a balance between P and I. I-gain is particularly useful when compensating for disturbances or modeling errors. Since our system is idealized, you will find that I-gain is not needed for good performance in simulation, but may be needed on the physical pendulum.

Keep iterating this process, tweaking your gains as you see how the system responds.

When you run this cell, note that we pass our `cart_pole_pid` and initial conditions to `solve_ivp`. (This stands for `solve_initial_value_problem`). This function solves our equations of motion given our initial conditions, just like Forward Euler. Except in this case, we use a fancier version, `RK45`.

In [ ]:
# --- YOUR CODE HERE ---
# Make a PID Controller
# Control gains (should be NEGATIVE)
kp = -10.0 # Should be negative
ki = -0.0 # Should be negative
kd = -0.0 # Should be negative

# Set the max actutor force limit in N
# It is 30 N on the robot, but you can change it as desired
F_max = 30
# --- END ---

pid_controller = PIDController(kp, ki, kd, dt)

# Initial conditions
x0 = 0 # initial cart position
x_dot0 = 0 # initial cart velocity
theta0 = np.radians(10) # initial pendulum angle
theta_dot0 = 0 # initial pendulum angular velocity
y0 = [x0, x_dot0, theta0, theta_dot0]

# Instantiate class
cart_pole_pid = CartPoleSystem(L, mA, mB, bt, br, g, IBxx, IByy, IBzz, pid_controller, F_max)

# Numerically integrate with RK45
sol_pid = solve_ivp(cart_pole_pid.cart_pole_dynamics, (0, t_end), y0, method='RK45', t_eval=t_eval, rtol=1e-6, atol=1e-9)

## Results: Plotting our Simulation Values!
Let's see how we did by plotting the state values over time. We'll plot the following:
- cart position $x$,
- cart velocity $v$,
- pole position $\theta$,
- pole angular velocity $\omega$.


For a successful controller, we want to see our values for $\theta$ stabilize and converge around $\theta=0^\circ$: the swing up position.

Do you notice any differences between this simulation and the step/impulse responses you were getting in the classical control activity? If so, what do you think could be causing these differences?

In [ ]:
# run the plotting code:
plot_cart_pole()

### Results: Running an Animation

In [ ]:
# --- PID ANIMATION ---
# Run animation
screen = pygame.display.set_mode((screen_width_px, screen_height_px))
np_frames = animate_system(sol_pid, len(t_eval), dt, animation_dt)

# Save animation
numpngw.write_apng(animation_path, np_frames, delay=animation_dt)
display(Image(filename=animation_path))

# STOP HERE UNTIL INSTRUCTED (PART 2... After LQR slides.)

## Cart-pole LQR Control

Recall from lecture that the linear quadratic regulator (LQR) finds the _optimal_ control policy that minimizes a quadratic cost specified by the user. That cost takes the following form:

$$
J_t = q_1 x_t^2 + q_2 v_t^2 + q_3 \theta_t^2 + q_4 \omega_t^2 + r_1 u_t^2
$$

In other words, crank up $q_1$ to penalize $x_t$ (cart position), or crank up $r_1$ to penalize control input $u_t$. Note that these gains are relative to each other - if you scale ALL of them up by the same amount (e.g., 10x), the system will behave identically.

### Your task:
Your task is to tune the cost matrices Q and R, specifying the cost that the LQR will minimize optimally. The cost matrices are currently:
```
Q = np.diag((0.0, 0.0, 0.0, 0.0))
R = np.diag([0.0])
```
(Where each diagonal entry in Q is $q_1, q_2, q_3, q_4$, and so on.)

Let's compare the PID performance to the _optimal_ LQR solution.

**Step 1.** Tune the gains, \\
**Step 2.** Compare your LQR to the PID gains tuned earlier in the cell.

First, let's calculate the state-space matrices $A$ and $B$ that define our system dynamics.

We're going to leverage symbolic python (`sympy`) to calculate the $A$ and $B$ matrices for us!

In [ ]:
### LQR IMPORTS ###
from scipy.linalg import solve_continuous_are
import sympy as sp

In [ ]:
# Let's let sympy do the work!

def get_state_space_matrices(L_num, mA_num, mB_num, bt_num, br_num, g_num, IBxx_num, IByy_num, IBzz_num):
    """Get the A, B matrices using symbolic computations"""
    # Define symbolic variables for the state
    x, qB, dx, dqB, ddx, ddqB = sp.symbols('x theta xdot thetadot xddot thetaddot')

    # Define symbolic variables for the physical system
    F, L, mA, mB, bt, br, g, IBxx, IByy, IBzz = sp.symbols('F L mA mB bt br g, IBxx, IByy, IBzz')

    # Define state and control matrices
    state = sp.Matrix([x, dx, qB, dqB])
    controls = sp.Matrix([F])

    # Translational equation of motion
    ddx = -0.5*(2*L*mB*sp.cos(qB)*(g*L*mB*sp.sin(qB)-2*br*dqB)+(4*IByy+mB*L**2)
          *(2*bt*dx-2*F-L*mB*sp.sin(qB)*dqB**2))/((mA+mB)*(4*IByy+mB*L**2)
          -L**2*mB**2*sp.cos(qB)**2)
    # Rotational equation of motion
    ddqB = (2*(mA+mB)*(g*L*mB*sp.sin(qB)-2*br*dqB)+L*mB*sp.cos(qB)
          *(2*bt*dx-2*F-L*mB*sp.sin(qB)*dqB**2))/((mA+mB)*(4*IByy+mB*L**2)
          -L**2*mB**2*sp.cos(qB)**2)

    f = sp.Matrix([
        dx,
        ddx,
        dqB,
        ddqB,
    ])

    # Compute the Jacobian matrices
    A_sym = f.jacobian(state)
    B_sym = f.jacobian(controls)

    # Substitute the values into the expression for A and B
    values = {
        g: g_num,
        bt: bt_num,
        br: br_num,
        mA: mA_num,
        mB: mB_num,
        L: L_num,
        IBxx: IBxx_num,
        IByy: IByy_num,
        IBzz: IBzz_num,
        x: 0,
        dx: 0,
        qB: 0,
        dqB: 0,
    }

    A = A_sym.subs(values)
    B = B_sym.subs(values)
    # Convert to NumPy arrays
    A_np = np.array(A).astype(np.float64)
    B_np = np.array(B).astype(np.float64)

    return A_np, B_np

# Example usage
A, B = get_state_space_matrices(L, mA, mB, bt, br, g, IBxx, IByy, IBzz)
print("These are the state-space matrices of the cart-pole system:")
print("A matrix (dynamics):")
print(A)
print("B matrix (control):")
print(B)

Next, we'll create an LQRController Class.

The LQR Controller is created with $A$ (dynamics matrix), $B$ (control matrix), $Q$ (state cost matrix), and $R$ (input cost matrix).

From those 4 inputs, we calculate $K$: our magical gain matrix that is optimal with respect to the costs $Q$ and $R$.

Our control input is simply $F = K e$ (where $e = x_{sp} - x$).

In [ ]:
# LQR controller
class LQRController:
  """Basic LQR Controller Class"""
  def __init__(self, A: np.ndarray, B: np.ndarray, Q: np.ndarray, R: np.ndarray) -> None:
    """Solve the Algebraic Ricatti Equation for K"""
    P = solve_continuous_are(A, B, Q, R)
    # Compute the LQR gain matrix K
    self.K = np.linalg.inv(R) @ (B.T @ P)
    print("K: ", self.K)

  def update(self, setpoint: np.ndarray, state: np.ndarray):
    """Calculate the input from the error"""
    error = setpoint - state
    return float(self.K @ error)

Finally, we create our LQRController and apply it to the CartPoleSystem.

In [ ]:
# DEMO: Make an LQR Controller
# LQR Controller Inputs
A, B = get_state_space_matrices(L, mA, mB, bt, br, g, IBxx, IByy, IBzz)
# --- YOUR CODE HERE: CHANGE THESE GAINS ---
Q = np.diag((0.0, 0.0, 0.0, 0.0))
R = np.diag([0.0])
# --- END ---
lqr_controller = LQRController(A, B, Q, R)

cart_pole_lqr = CartPoleSystem(L, mA, mB, bt, br, g, IBxx, IByy, IBzz, lqr_controller, F_max)

# Numerically integrate with RK45
sol_lqr = solve_ivp(cart_pole_lqr.cart_pole_dynamics, (0, t_end), y0, method='RK45', t_eval=t_eval, rtol=1e-6, atol=1e-9)

In this last cell, we plot the solutions to both the LQR controller and the PID controller. What do you notice?

1. If you chose your gains correctly, the LQR controller is able to stabilize all state variables: $x$, $v$, $\theta$, and $\omega$. (In contrast, PID can only stabilize $\theta$.)
2. If you tuned it properly, the PID controller actually does a better job stabilizing $\theta$: driving $\theta$ to zero faster and with less overshoot! Why is this the case? (Answer: While PID only has one objective, LQR needs to compromise in order to minimize cost and drive all state variables to zero.)


In [ ]:
plot_cart_pole(PLOT_LQR = True)

In [ ]:
# --- LQR ANIMATION ---
# Run animation
screen = pygame.display.set_mode((screen_width_px, screen_height_px))
np_frames = animate_system(sol_lqr, len(t_eval), dt, animation_dt)

# Save animation
numpngw.write_apng(animation_path, np_frames, delay=animation_dt)
display(Image(filename=animation_path))

### Cost Comparison

Finally, let's compare the "cost" between LQR and PID. Remember, LQR is the _optimal_ control policy that minimizes the cost -- there can be no better!

Recall that our cost is defined as the following:
$$
    J_{\infty} =\int_0^\infty \mathbf{x}_t^T Q \mathbf{x}_t + \mathbf{u}_t^T R \mathbf{u}_t \; dt
$$

As such, the LQR policy will seek to minimize the cumulative cost (state error and control input) as weighted by the $Q$ and $R$ matrices.

In [ ]:
# Get the control inputs from the history
ctrl_t, ctrl_F = zip(*cart_pole_pid.F_history)
ctrl_t_lqr, ctrl_F_lqr = zip(*cart_pole_lqr.F_history)
# Resample them to match the size of the solution vector
ctrl_F_at_eval = interp1d(ctrl_t, ctrl_F, kind='nearest', fill_value="extrapolate")(sol_pid.t).squeeze()
ctrl_F_lqr_at_eval = interp1d(ctrl_t_lqr, ctrl_F_lqr, kind='nearest', fill_value="extrapolate")(sol_lqr.t).squeeze()

# Compute the quadratic cost at each time step (using broadcasting)
pid_state_cost = np.einsum('ij,ik,kj->j', sol_pid.y, Q, sol_pid.y)
lqr_state_cost = np.einsum('ij,ik,kj->j', sol_lqr.y, Q, sol_lqr.y)
# Compute the control cost at each timestep
pid_control_cost = R.item()*ctrl_F_at_eval**2
lqr_control_cost = R.item()*ctrl_F_lqr_at_eval**2

# Compute the cumulative cost over time
pid_cumulative_cost = np.cumsum(pid_state_cost + pid_control_cost)
lqr_cumulative_cost = np.cumsum(lqr_state_cost + lqr_control_cost)

# Plot the cumulative cost
plt.figure()
plt.plot(sol_pid.t, pid_cumulative_cost, label='PID')
plt.plot(sol_lqr.t, lqr_cumulative_cost, label='LQR')
plt.legend()
plt.title(f'Cart Pole: Cumulative Cost J')
plt.xlabel(f't (seconds)')
plt.ylabel(f'Cumulative Cost J')